In [12]:
%%hide_code
# Initialization - installs & imports 
import os
import xml.etree.ElementTree as ET
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

!pip install voila
!pip install openpyxl

!jupyter labextension update --all
!jupyter nbextension update --all
!pip install --upgrade ipywidgets


UsageError: Cell magic `%%hide_code` not found.


In [2]:
# Display the intro screen with Voila components
from IPython.display import display, HTML

def display_intro_screen(app_name):
    intro_screen = HTML(f'''
        <div style="text-align: center;">
            <h1 style="font-size: 36px;">{app_name}</h1>
            <h2 style="font-size: 24px;">Welcome to the XML Processing App!</h2>
        </div>
    ''')
    display(intro_screen)

app_name = "PC's XML Data Processing App"
display_intro_screen(app_name)



In [ ]:
# Directory containing XML files
xml_directory = '/home/jovyan/workspace/XML_files'

# Load the Excel file
xls_file = '/home/jovyan/workspace/PC INVENTORY 2024.xlsx'
xls_data = pd.read_excel(xls_file)

In [9]:
from IPython.display import display
import ipywidgets as widgets

# Button to launch XML files processing
process_button = widgets.Button(description='Process XML Files', 
                                layout=widgets.Layout(margin='auto', width='200px'),
                                button_style='info')
output = widgets.Output(layout=widgets.Layout(display='flex', justify_content='center'))

def on_button_clicked(b):
    with output:
        # Codice per elaborare i file XML e generare il risultato
        # ...
        print("Processing completed.")

process_button.on_click(on_button_clicked)

# CSS styling
style = """
<style>
.widget-button {
    animation: blink 1s linear infinite;
}
@keyframes blink {
    50% {
        background-color: #2196F3;
    }
}
</style>
"""

# Display the button and output
display(HTML(style))
display(widgets.VBox([process_button, output]))



In [7]:
# Initialize an empty DataFrame for the results
result_df = pd.DataFrame(columns=['HOSTNAME', 'MAC Address', 'IP', 'OS', 'Antivirus', 'SERIALE CESPITI', 'Reparto', 'MODELLO PC', 'S/N PC', 'Note'])

# Function to extract OS information
def extract_os_info(element):
    os_info = {}
    for entry in element.findall('./entry'):
        title = entry.find('entry[@title="Model"]')
        if title is not None:
            os_info['Model'] = title.get('value')
        title = entry.find('entry[@title="Edition"]')
        if title is not None:
            os_info['Edition'] = title.get('value')
        title = entry.find('entry[@title="Architecture"]')
        if title is not None:
            os_info['Architecture'] = title.get('value')
    return os_info

# Function to extract Antivirus information
def extract_antivirus_info(element):
    antivirus_info = {}
    for entry in element.findall('./entry'):
        if entry.get('title') == 'Company Name':
            antivirus_info['Company Name'] = entry.get('value')
        elif entry.get('title') == 'Product Version':
            antivirus_info['Product Version'] = entry.get('value')
        elif entry.get('title') == 'Display Name':
            antivirus_info['Display Name'] = entry.get('value')
    return antivirus_info

# Function to extract MAC Address
def extract_mac_address(element):
    for section in element.findall('.//section'):
        if section.get('title') == 'Adapters List':
            for adapter_section in section.findall('.//section'):
                if adapter_section.get('title') == 'Enabled':
                    for entry in adapter_section.findall('.//entry'):
                        if entry.get('title') == 'MAC Address':
                            return entry.get('value')
    return None

# Recursive function to extract data
def extract_entries(element, main_section=None, section_id=None, parent_section=None):
    data = []
    if element.tag == 'entry' and element.get('title') == 'IP Address':
        data.append({
            'main_section': main_section,
            'section_id': section_id,
            'parent_section': parent_section,
            'entry_title': element.get('title'),
            'entry_value': element.get('value')
        })
    for child in element:
        if child.tag == 'mainsection':
            main_section = child.get('title')
            if main_section == 'Operating System':
                os_info = extract_os_info(child)
                data.append({
                    'main_section': main_section,
                    'section_id': section_id,
                    'parent_section': parent_section,
                    'entry_title': 'Operating System',
                    'entry_value': f"{os_info.get('Model', '')} {os_info.get('Edition', '')} {os_info.get('Architecture', '')}".strip()
                })
            data.extend(extract_entries(child, main_section, section_id))
        elif child.tag == 'section':
            section_id = child.get('id')
            if child.get('title') == 'Antivirus':
                antivirus_info = extract_antivirus_info(child)
                if not antivirus_info:
                    for antivirus_section in child.findall('./section'):
                        antivirus_info = extract_antivirus_info(antivirus_section)
                        if antivirus_info:
                            break
                if antivirus_info:
                    data.append({
                        'main_section': main_section,
                        'section_id': section_id,
                        'parent_section': parent_section,
                        'entry_title': 'Antivirus',
                        'entry_value': f"{antivirus_info.get('Company Name', '')} {antivirus_info.get('Product Version', '')} {antivirus_info.get('Display Name', '')}".strip()
                    })
            data.extend(extract_entries(child, main_section, section_id, child.get('title')))
        else:
            data.extend(extract_entries(child, main_section, section_id, parent_section))
    return data

In [5]:
# Iterate over the rows in the Excel file
for index, row in xls_data.iterrows():
    hostname = row['HOSTNAME']
    xml_file = os.path.join(xml_directory, f"{hostname}.xml")
    
    if os.path.exists(xml_file):
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Start extraction from the root element
        data = extract_entries(root)

        # Extract MAC Address
        mac_address = extract_mac_address(root)

        # Convert the data into a pandas DataFrame
        df = pd.DataFrame(data)

        # Extract IP Address, Operating System, and Antivirus information
        ip_address = df.loc[df['entry_title'] == 'IP Address', 'entry_value'].values[0] if 'IP Address' in df['entry_title'].values else None
        os_value = df.loc[df['entry_title'] == 'Operating System', 'entry_value'].values[0] if 'Operating System' in df['entry_title'].values else 'NaN'
        antivirus_value = df.loc[df['entry_title'] == 'Antivirus', 'entry_value'].values[0] if 'Antivirus' in df['entry_title'].values else 'NaN'

        result_row = {
            'HOSTNAME': hostname,
            'MAC Address': mac_address,
            'IP': ip_address,
            'OS': os_value,
            'Antivirus': antivirus_value,
            'SERIALE CESPITI': row['SERIALE CESPITI'],
            'Reparto': row['Reparto'],
            'MODELLO PC': row['MODELLO PC'],
            'S/N PC': row['S/N PC'],
            'Note': row['Note']
        }

        # Create a DataFrame with a single row of data
        result_row_df = pd.DataFrame([result_row])

        # Append the row to the result DataFrame
        result_df = pd.concat([result_df, result_row_df], ignore_index=True)

In [6]:
# Display the summary of the result DataFrame
summary = result_df.describe()
print(summary)

# Save the result DataFrame to a CSV file with semicolon delimiter
result_df.to_csv('output.csv', sep=';', index=False)

        HOSTNAME        MAC Address            IP                     OS  \
count         10                 10            10                     10   
unique        10                 10            10                      4   
top     TCEWORK2  6C-02-E0-6D-7D-CD  10.100.8.193  Windows 10 Pro 64-bit   
freq           1                  1             1                      4   

                      Antivirus           SERIALE CESPITI Reparto MODELLO PC  \
count                        10                        10       0          0   
unique                        3                        10       0          0   
top     Symantec 14.3.1148.0100  e280699500007003a338a5ff     NaN        NaN   
freq                          5                         1     NaN        NaN   

            S/N PC Note  
count           10    0  
unique          10    0  
top     czc1418bxj  NaN  
freq             1  NaN  
